<a href="https://colab.research.google.com/github/mhy1264/taipower/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install autoKeras
!pip install scikit-learn

In [1]:
import autokeras as ak
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import weather

Using TensorFlow backend


In [2]:
def get_obv_station(gen_station: str) -> str:
    obv_station = pd.read_csv("./data/mini_dist2.csv")
    df = obv_station[obv_station['gen_station'].astype(str).str.contains("{}".format(gen_station))]
    return df['obv_station'].tolist()[0]

In [8]:

solar_daily = pd.read_csv("./data/solar_daily.csv")
station = pd.read_csv("./data/station.csv")


for row in range(1,solar_daily.shape[1]-2):
    df = solar_daily.iloc[:, [row, -1]]
    current_station = df.columns.tolist()[0]
    print("{}".format(current_station))
    try:
        obv_station = get_obv_station(current_station)
        station_weather = weather.get_month_data(obv_station, 2017, 1)
        merged_df = pd.concat([df, station_weather], axis=1, join="inner")
    except Exception :
        print ("error occur")





金門金沙光電
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=C0W140&stname=%25E9%2587%2591%25E6%25B2%2599&datepicker=2017-01&altitude=21.0m#
台中電廠光電
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=C0FA40&stname=%25E8%2587%25BA%25E4%25B8%25AD%25E9%259B%25BB%25E5%25BB%25A0&datepicker=2017-01&altitude=25.0m#
興達電廠光電
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=C0V880&stname=%25E8%2588%2588%25E9%2581%2594&datepicker=2017-01&altitude=19.0m#
嘉義民雄光電
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=467480&stname=%25E5%2598%2589%25E7%25BE%25A9&datepicker=2017-01&altitude=26.9m#
中部儲運中心光電
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=C0F9L0&stname=%25E5%2590%258E%25E9%2587%258C&datepicker=2017-01&altitude=253.0m#
東勢新伯公光電
https://e-service.cwb.gov.tw/HistoryDataQuery/MonthDat